In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys
from unittest.mock import AsyncMock, Mock, patch

import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn.functional as F
from dotenv import load_dotenv
from matplotlib.colors import ListedColormap

# Make sure your dotenv file has the following defined:
load_dotenv()
GIT_FOLDER = os.environ['GIT_FOLDER']
TITILER_URL = os.environ['TITILER_URL']
TITILER_API_KEY = os.environ['TITILER_API_KEY']
API_KEY = os.environ['API_KEY']
MODEL_PATH_LOCAL = os.environ['MODEL_PATH_LOCAL_UNET']
# MODEL_PATH_LOCAL = os.environ['MODEL_PATH_LOCAL_MASK']

if not (GIT_FOLDER and TITILER_URL and TITILER_API_KEY and MODEL_PATH_LOCAL):
    print("ERRROR: Failed to find all the necessary environment variables!!!")
    # Note, you must restart the kernel if you want to load new environment variables

if GIT_FOLDER not in sys.path:
    sys.path.append(GIT_FOLDER)
print(sys.path)

In [ ]:
from cerulean_cloud.models import get_model, memfile_gtiff
from cerulean_cloud.tiling import TMS, offset_bounds_from_base_tiles
from cerulean_cloud.titiler_client import TitilerClient
from cerulean_cloud.cloud_run_orchestrator.clients import img_array_to_b64_image
from cerulean_cloud.cloud_run_orchestrator.schema import OrchestratorInput
from cerulean_cloud.cloud_run_orchestrator.handler import _orchestrate, get_tiler, get_titiler_client, get_roda_sentinelhub_client, get_database_engine
from cerulean_cloud.cloud_run_offset_tiles.schema import InferenceInput, PredictPayload
from cerulean_cloud.cloud_run_offset_tiles.handler import predict

In [ ]:
fastaiunet = dict(
    type="FASTAIUNET",
    file_path="",#"experiments/2024_03_06_18_14_31_7cls_rn101_pr256_z9_fastai_baseline_noamb/tracing_cpu_model.pt",
    layers=["VV"],
    cls_map={
        0: "BACKGROUND",
        1: "INFRA",
        2: "NATURAL",
        3: "VESSEL",
    },  # inference_idx maps to class table
    name="local test",
    tile_width_m=40844,  # Used to calculate zoom
    tile_width_px=512,  # Used to calculate scale
    epochs=237,
    thresholds={
        "poly_nms_thresh": 0.1,
        "pixel_nms_thresh": 0.1,
        "bbox_score_thresh": 0.01,
        "poly_score_thresh": 0.3,
        "pixel_score_thresh": 0.8,
        "groundtruth_dice_thresh": 0.0,
    },
    backbone_size=18,
    # pixel_f1=0.0, # TODO CALCULATE
    # instance_f1=0.0, # TODO CALCULATE
)

maskrcnn = dict(
    type="MASKRCNN",
    file_path="",#"experiments/2023_10_05_02_22_46_4cls_rnxt101_pr512_px1024_680min_maskrcnn_wd01/scripting_cpu_model.pt",
    layers=["VV", "ALL_255", "VESSEL"],
    cls_map={
        0: "BACKGROUND",
        1: "INFRA",
        2: "NATURAL",
        3: "VESSEL",
    },  # inference_idx maps to class table
    name="ResNext 101 hires56",
    tile_width_m=40844,
    tile_width_px=512,
    epochs=122,
    thresholds={
        "poly_nms_thresh": 0.2,
        'pixel_nms_thresh': 0.4,
        'bbox_score_thresh': 0.3,
        'poly_score_thresh': 0.1,
        'pixel_score_thresh': 0.5,
        'groundtruth_dice_thresh': 0.0
        },
    backbone_size=101,
    pixel_f1=0.461,
    instance_f1=0.47,
)

model_dict_predefined=maskrcnn if "maskrcnn" in MODEL_PATH_LOCAL else fastaiunet

In [ ]:
async def get_titiler_client_and_offset_tiles(sentinel_scene, offset=.33):
    payload = OrchestratorInput(**sentinel_scene)
    TitilerClient_url = os.getenv('TITILER_URL')
    titiler_client = TitilerClient(url=TitilerClient_url)
    scene_bounds = await titiler_client.get_bounds(payload.sceneid)
    tiler = TMS
    base_tiles = list(tiler.tiles(*scene_bounds, [payload.zoom], truncate=False))
    offset_tile_bounds = offset_bounds_from_base_tiles(base_tiles, offset_amount=offset)
    return titiler_client, offset_tile_bounds

In [ ]:
# EXPLORE TILES FROM A GIVEN SCENE_ID
offset=.33*0
titler_client , tile_bounds =  await get_titiler_client_and_offset_tiles(test_scene,offset=offset)
for i, tile in enumerate(tile_bounds):
    plt.imshow((await titler_client.get_offset_tile(scene_id, *tile,height=512,width=512)).transpose(2,0,1)[0], cmap="gray")
    plt.title(str(i))
    plt.show()

In [ ]:
scene_id = "S1A_IW_GRDH_1SDV_20230618T232014_20230618T232039_049047_05E5E0_718C"
test_scene = {"sceneid": scene_id , "zoom":9, "scale":2}

slick_tiles = {
    0: [17, 25],
    .33: [18, 26],
    .66: [18, 19],
    } 

In [ ]:
scene_id = "S1A_IW_GRDH_1SDV_20230523T224049_20230523T224114_048667_05DA7A_91D1"
test_scene = {"sceneid": scene_id , "zoom":9, "scale":2}

slick_tiles = {
    0: [44, 45, 51, 52, 53],
    .33: [37, 38, 44, 45, 46, 52, 53, 54],
    .66: [37, 38, 45, 46, 47],
    } 

In [ ]:
scene_id = "S1A_IW_GRDH_1SDV_20200729T095401_20200729T095430_033668_03E6EE_2611"
test_scene = {"sceneid": scene_id , "zoom":9, "scale":2}

slick_tiles = {
    0: [18, 19, 25, 26, 32, 33],
    .33: [18, 19, 25, 26, 33],
    .66: [11,12, 18, 19, 25, 26],
    } 

In [ ]:
scene_id = "S1A_IW_GRDH_1SDV_20230318T175405_20230318T175430_047702_05BAED_22A9"
test_scene = {"sceneid": scene_id , "zoom":9, "scale":2}

slick_tiles = {
    0: [26, 28, 34, 35, 36, 43],
    # .33: [],
    # .66: [],
    } 

In [ ]:
scene_id = "S1A_IW_GRDH_1SDV_20201114T034910_20201114T034935_035239_041D79_AA81"
test_scene = {"sceneid": scene_id , "zoom":9, "scale":2}

slick_tiles = {
    0: [34, 42, 51, 59, 67, 75],
    .33: [34, 35, 43, 51, 59, 60, 67, 68],
    .66: [35, 43, 51, 60, 68],
    } 

In [ ]:
scene_id = "S1B_IW_GRDH_1SDV_20201023T170409_20201023T170433_023943_02D81C_C8C1"
test_scene = {"sceneid": scene_id , "zoom":9, "scale":2}

slick_tiles = {
    0: [18], # , 25, 26
    .33: [18, 19],
    .66: [12, 19],
    } 

In [ ]:
scene_id = "S1A_IW_GRDH_1SDV_20240204T184243_20240204T184308_052413_0656A2_1B88"
test_scene = {"sceneid": scene_id , "zoom":9, "scale":2}

slick_tiles = {
    0: [44, 52],
    .33: [37, 45],
    .66: [37, 45],
    } 

In [ ]:
scene_id = "S1A_IW_GRDH_1SDV_20201009T201613_20201009T201642_034724_040B9A_67CD"
test_scene = {"sceneid": scene_id , "zoom":9, "scale":2}

slick_tiles = {
    0: [44, 45], 
    0.33: [37, 45], 
    0.66: [37, 45], 
    } 

In [ ]:
model = get_model(model_dict_predefined,model_path_local=MODEL_PATH_LOCAL)
out_stack = []
bounds_stack = []

for o in slick_tiles.keys():
    print("offset", o)
    titler_client , tile_bounds =  await get_titiler_client_and_offset_tiles(test_scene,offset=o)
    inf_stack = []
    vv_stack = []
    for i in slick_tiles[o]:
        print("bounds", i)
        vv = (await titler_client.get_offset_tile(scene_id, *tile_bounds[i],height=512,width=512)).transpose(2,0,1)[0]
        vv_stack.append(vv)
        inf = InferenceInput(image=img_array_to_b64_image(np.array([vv]*len(model_dict_predefined["layers"]))))
        inf_stack.append(inf)
    out = model.predict(inf_stack)
    out_stack.append(out)
    bounds_stack.append([tile_bounds[i] for i in slick_tiles[o]])


In [ ]:
model = get_model(model_dict_predefined,model_path_local=MODEL_PATH_LOCAL)

fc_stack = []
for o, b in zip(out_stack, bounds_stack):
    fc = model.postprocess_tileset([o], b)
    fc_stack.append(fc)

# Ensemble the FCs together
fc_f = model.nms_feature_reduction(
    features=fc_stack, 
    min_overlaps_to_keep=(1 if len(slick_tiles)>1 else 0)
)

In [ ]:
import geopandas as gpd

# PLOT EVERYTHING
gdf_f = gpd.GeoDataFrame.from_features(fc_f["features"])

fig, ax = plt.subplots(figsize=(20, 20))
colors = plt.cm.get_cmap('jet', len(gdf_f))
for i, g in gdf_f.iterrows():
    gdf_f.loc[[i]].plot(ax=ax, color=colors(i), alpha=0.75)
    ax.plot([], [], color=colors(i), label=f"{i} {model_dict_predefined['cls_map'][g['inf_idx']]}: {round(g['machine_confidence'],2)}")
ax.legend(title="Machine Confidence", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()

gdf_f


In [ ]:
from rasterio.plot import show
from rasterio.merge import merge as rio_merge

ds_tiles = [
    memfile_gtiff(nparray=array, bounds=bounds).open()
    for array, bounds in zip(vv_stack, bounds_stack[-1])
]

# Merge datasets
scene_array, transform = rio_merge(ds_tiles)

# Plotting the merged image
fig, ax = plt.subplots(figsize=(20, 20))  # You can adjust the size as needed
show(scene_array[0], transform=transform, ax=ax)  # Assuming merged array is the first band
gdf_f.plot(ax=ax, alpha=0.3, edgecolor='red', facecolor='none')  # Adjust alpha for transparency
ax.set_title("Stitched Image")
plt.show()